###### Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Daniel Koehn based on Jupyter notebooks by Marc Spiegelman [Dynamical Systems APMA 4101](https://github.com/mspieg/dynamical-systems) and  Kyle Mandli from his course [Introduction to numerical methods](https://github.com/mandli/intro-numerical-methods), notebook style sheet by L.A. Barba, N.C. Clementi [Engineering Computations](https://github.com/engineersCode)

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Exploring the Lorenz Equations

The Lorenz Equations are a 3-D dynamical system that is a simplified model of Rayleigh-Benard thermal convection.  They are derived and described in detail in Edward Lorenz' 1963 paper [Deterministic Nonperiodic Flow](http://journals.ametsoc.org/doi/pdf/10.1175/1520-0469%281963%29020%3C0130%3ADNF%3E2.0.CO%3B2)  in the Journal of Atmospheric Science.  In their classical form they can be written

\begin{equation}
\begin{split}
\frac{\partial X}{\partial t} &= \sigma( Y - X)\\
\frac{\partial Y}{\partial t} &= rX - Y - XZ \\
\frac{\partial Z}{\partial t} &= XY -b Z
\end{split}
\tag{1}
\end{equation}

where $\sigma$ is the "Prandtl number",  $r = \mathrm{Ra}/\mathrm{Ra}_c$ is a scaled "Rayleigh number" and $b$ is a parameter that is related to the the aspect ratio of a convecting cell in the original derivation.

Here, $X(t)$, $Y(t)$ and $Z(t)$ are the time dependent amplitudes of the streamfunction and temperature fields, expanded in a highly truncated Fourier Series where the streamfunction contains one cellular mode

$$
    \psi(x,z,t) = X(t)\sin(a\pi x)\sin(\pi z)
$$

and temperature has two modes

$$
    \theta(x,z,t) = Y(t)\cos(a\pi x)\sin(\pi z) - Z(t)\sin(2\pi z)
$$

This Jupyter notebook, will provide some simple python routines for numerical integration and visualization of the Lorenz Equations.

## Numerical solution of the Lorenz Equations

We have to solve the uncoupled ordinary differential equations (1) using the finite difference method introduced in [this lecture](https://nbviewer.jupyter.org/github/daniel-koehn/Differential-equations-earth-system/blob/master/02_finite_difference_intro/1_fd_intro.ipynb).

The approach is similar to the one used in [Exercise: How to sail without wind](https://nbviewer.jupyter.org/github/daniel-koehn/Differential-equations-earth-system/blob/master/02_finite_difference_intro/3_fd_ODE_example_sailing_wo_wind.ipynb), except that eqs.(1) are coupled ordinary differential equations, we have an additional differential equation and the RHS are more complex. 

Approximating the temporal derivatives in eqs. (1) using the **backward FD operator** 

\begin{equation}
\frac{df}{dt} = \frac{f(t)-f(t-dt)}{dt} \notag
\end{equation}

with the time sample interval $dt$ leads to 

\begin{equation}
\begin{split}
\frac{X(t)-X(t-dt)}{dt} &= \sigma(Y - X)\\
\frac{Y(t)-Y(t-dt)}{dt} &= rX - Y - XZ\\
\frac{Y(t)-Y(t-dt)}{dt} &= XY -b Z\\
\end{split}
\notag
\end{equation}

After solving for $X(t), Y(t), Z(t)$, we get the **explicit time integration scheme** for the Lorenz equations:

\begin{equation}
\begin{split}
X(t) &= X(t-dt) + dt\; \sigma(Y - X)\\
Y(t) &= Y(t-dt) + dt\; (rX - Y - XZ)\\
Z(t) &= Z(t-dt) + dt\; (XY -b Z)\\
\end{split}
\notag
\end{equation}

and by introducing a temporal dicretization $t^n = n * dt$ with $n \in [0,1,...,nt]$, where $nt$ denotes the maximum time steps, the final FD code becomes:

\begin{equation}
\begin{split}
X^{n} &= X^{n-1} + dt\; \sigma(Y^{n-1} - X^{n-1})\\
Y^{n} &= Y^{n-1} + dt\; (rX^{n-1} - Y^{n-1} - X^{n-1}Z^{n-1})\\
Z^{n} &= Z^{n-1} + dt\; (X^{n-1}Y^{n-1} - b Z^{n-1})\\
\end{split}
\tag{2}
\end{equation}

The Python implementation is quite straightforward, because we can reuse some old codes ...

##### Exercise 1

Finish the function `Lorenz`, which computes and returns the RHS of eqs. (1) for a given $X$, $Y$, $Z$.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def Lorenz(X,Y,Z,sigma,r,b):
    
  '''
  Returns the RHS of the Lorenz equations
  '''

  # ADD RHS OF LORENZ EQUATIONS (1) HERE!
  X_dot_rhs =
  Y_dot_rhs =
  Z_dot_rhs =

  # return the state derivatives
  return X_dot_rhs, Y_dot_rhs, Z_dot_rhs

Next, we write the function to solve the Lorenz equation `SolveLorenz` based on the `sailing_boring` code from the [Exercise: How to sail without wind](https://nbviewer.jupyter.org/github/daniel-koehn/Differential-equations-earth-system/blob/master/02_finite_difference_intro/3_fd_ODE_example_sailing_wo_wind.ipynb)

##### Exercise 2

Finish the FD-code implementation `SolveLorenz`

In [ ]:
def SolveLorenz(tmax, dt, X0, Y0, Z0, sigma=10.,r=28.,b=8./3.0):
    
    '''
    Integrate the Lorenz equations from initial condition (X0,Y0,Z0)^T at t=0 
    for parameters sigma, r, b
    
    Returns: X, Y, Z, time
    '''
    
    # Compute number of time steps based on tmax and dt
    nt = (int)(tmax/dt)
    
    # vectors for storage of X, Y, Z positions and time t
    X  = np.zeros(nt + 1)
    Y  = np.zeros(nt + 1)
    Z  = np.zeros(nt + 1)
    t  = np.zeros(nt + 1)
    
    # define initial position and time
    X[0] = X0
    Y[0] = Y0
    Z[0] = Z0
    
    # start time stepping over time samples n
    for n in range(1,nt + 1):
    
        # compute RHS of Lorenz eqs. (1) at current position (X,Y,Z)^T
        X_dot_rhs, Y_dot_rhs, Z_dot_rhs = Lorenz(X[n-1],Y[n-1],Z[n-1],sigma,r,b)
    
        # compute new position using FD approximation of time derivative
        # ADD FD SCHEME OF THE LORENZ EQS. HERE!
        X[n] = 
        Y[n] = 
        Z[n] =
        t[n] = n * dt

    return X, Y, Z, t

Finally, we create a function to plot the solution (X,Y,Z)^T of the Lorenz eqs. ...

In [ ]:
def PlotLorenzXvT(X,Y,Z,t,sigma,r,b):
    
    '''
    Create time series plots of solutions of the Lorenz equations X(t),Y(t),Z(t)
    '''

    plt.figure()
    ax = plt.subplot(111)
    ax.plot(t,X,'r',label='X')
    ax.plot(t,Y,'g',label='Y')
    ax.plot(t,Z,'b',label='Z')
    ax.set_xlabel('time t')
    plt.title('Lorenz Equations: $\sigma=${}, $r=${}, $b=${}'.format(sigma,r,b))
    # Shrink current axis's height by 10% on the bottom
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])

    # Put a legend below current axis
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),ncol=3)
    plt.show()

... and a function to plot the trajectory in the **phase space portrait**:

In [ ]:
def PlotLorenz3D(X,Y,Z,sigma,r,b):
    '''
    Show 3-D Phase portrait using mplot3D
    '''
    # do some fancy 3D plotting
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot(X,Y,Z)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.title('Lorenz Equations: $\sigma=${}, $r=${}, $b=${}'.format(sigma,r,b))
    plt.show()

##### Exercise 3

Solve the Lorenz equations for a Prandtl number $\sigma=10$, $b=8/3$ and a scaled Rayleigh number $r=0.5$, starting from the initial condition ${\bf{X_0}}=(X_0,Y_0,Z_0)^T=(2,3,4)^T$. Plot the temporal evolution and 3D phase potrait of the solution $(X(t),Y(t),Z(t))^T$. Mark the fix points, you derived in [Stationary Solutions of Time-Dependent Problems](http://nbviewer.ipython.org/urls/github.com/daniel-koehn/Differential-equations-earth-system/tree/master/03_Lorenz_equations/02_Stationary_solutions_of_DE.ipynb) in the 3D phase portrait. Describe and interpret the results.

In [ ]:
# SET THE PARAMETERS HERE!
sigma= 
b = 

# SET THE INITIAL CONDITIONS HERE!
X0 = 
Y0 = 
Z0 =

# Set maximum integration time and sample interval dt
tmax = 30
dt = 0.01

# SET THE RAYLEIGH NUMBER HERE!
r =

# Solve the Equations
X, Y, Z, t = SolveLorenz(tmax, dt, X0, Y0, Z0, sigma,r,b)

# and Visualize as a time series
PlotLorenzXvT(X,Y,Z,t,sigma,r,b)

# and as a 3-D phase portrait
PlotLorenz3D(X,Y,Z,sigma,r,b)

##### Exercise 4

Solve the Lorenz equations for a Prandtl number $\sigma=10$, $b=8/3$ and a scaled Rayleigh number $r=0.5$, starting from the initial condition ${\bf{X_0}}=(X_0,Y_0,Z_0)^T=(2,3,4)^T$. Plot the temporal evolution and 3D phase potrait of the solution $(X(t),Y(t),Z(t))^T$. Mark the fix points, you derived in [Stationary Solutions of Time-Dependent Problems](http://nbviewer.ipython.org/urls/github.com/daniel-koehn/Differential-equations-earth-system/tree/master/03_Lorenz_equations/02_Stationary_solutions_of_DE.ipynb) in the 3D phase portrait. Describe and interpret the results.

In [ ]:
# SET THE PARAMETERS HERE!
sigma= 
b = 

# SET THE INITIAL CONDITIONS HERE!
X0 = 
Y0 = 
Z0 =

# Set maximum integration time and sample interval dt
tmax = 30
dt = 0.01

# SET THE RAYLEIGH NUMBER HERE!
r =

# Solve the Equations
X, Y, Z, t = SolveLorenz(tmax, dt, X0, Y0, Z0, sigma,r,b)

# and Visualize as a time series
PlotLorenzXvT(X,Y,Z,t,sigma,r,b)

# and as a 3-D phase portrait
PlotLorenz3D(X,Y,Z,sigma,r,b)

##### Exercise 5

Solve the Lorenz equations again for a Prandtl number $\sigma=10$, $b=8/3$ and a scaled Rayleigh number $r=0.5$. However, starting from the initial condition ${\bf{X_0}}=(X_0,Y_0,Z_0)^T=(-2,-3,4)^T$. Plot the temporal evolution and 3D phase potrait of the solution $(X(t),Y(t),Z(t))^T$. Mark the fix points, you derived in [Stationary Solutions of Time-Dependent Problems](http://nbviewer.ipython.org/urls/github.com/daniel-koehn/Differential-equations-earth-system/tree/master/03_Lorenz_equations/02_Stationary_solutions_of_DE.ipynb) in the 3D phase portrait. Describe and interpret the results. How does the solution change compared to exercise 4?

In [ ]:
# SET THE PARAMETERS HERE!
sigma= 
b = 

# SET THE INITIAL CONDITIONS HERE!
X0 = 
Y0 = 
Z0 =

# Set maximum integration time and sample interval dt
tmax = 30
dt = 0.01

# SET THE RAYLEIGH NUMBER HERE!
r =

# Solve the Equations
X, Y, Z, t = SolveLorenz(tmax, dt, X0, Y0, Z0, sigma,r,b)

# and Visualize as a time series
PlotLorenzXvT(X,Y,Z,t,sigma,r,b)

# and as a 3-D phase portrait
PlotLorenz3D(X,Y,Z,sigma,r,b)

##### Exercise 6

Solve the Lorenz equations for a Prandtl number $\sigma=10$, $b=8/3$ and a scaled Rayleigh number $r=28$, starting from the initial condition ${\bf{X_0}}=(X_0,Y_0,Z_0)^T=(2,3,4)^T$. Plot the temporal evolution and 3D phase potrait of the solution $(X(t),Y(t),Z(t))^T$. Mark the fix points, you derived in [Stationary Solutions of Time-Dependent Problems](http://nbviewer.ipython.org/urls/github.com/daniel-koehn/Differential-equations-earth-system/tree/master/03_Lorenz_equations/02_Stationary_solutions_of_DE.ipynb) in the 3D phase portrait. Describe and interpret the results. Compare with the previous results.

In [ ]:
# SET THE PARAMETERS HERE!
sigma= 
b = 

# SET THE INITIAL CONDITIONS HERE!
X0 = 
Y0 = 
Z0 =

# Set maximum integration time and sample interval dt
tmax = 30
dt = 5e-4

# SET THE RAYLEIGH NUMBER HERE!
r =

# Solve the Equations
X, Y, Z, t = SolveLorenz(tmax, dt, X0, Y0, Z0, sigma,r,b)

# and Visualize as a time series
PlotLorenzXvT(X,Y,Z,t,sigma,r,b)

# and as a 3-D phase portrait
PlotLorenz3D(X,Y,Z,sigma,r,b)

##### Exercise 7

In his 1963 paper Lorenz also investigated the influence of small changes of the initial conditions on the long-term evolution of the thermal convection problem for large Rayleigh numbers. 

Solve the Lorenz equations for a Prandtl number $\sigma=10$, $b=8/3$ and a scaled Rayleigh number $r=28$, however starting from the initial condition ${\bf{X_0}}=(X_0,Y_0,Z_0)^T=(2,3.001,4)^T$. Plot the temporal evolution and compare with the solution of exercise 6. Describe and interpret the results.

Explain why Lorenz introduced the term **Butterfly effect** based on your results.

In [ ]:
# SET THE PARAMETERS HERE!
sigma= 
b = 

# SET THE INITIAL CONDITIONS HERE!
X0 = 
Y0 = 
Z0 =

# Set maximum integration time and sample interval dt
tmax = 30
dt = 5e-4

# SET THE RAYLEIGH NUMBER HERE!
r =

# Solve the Equations
X1, Y1, Z1, t = SolveLorenz(tmax, dt, X0, Y0, Z0, sigma,r,b)

# and Visualize differences as a time series
PlotLorenzXvT(X-X1,Y-Y1,Z-Z1,t,sigma,r,b)

# and Visualize as a time series
PlotLorenzXvT(X1,Y1,Z1,t,sigma,r,b)

# and Visualize as a time series
PlotLorenzXvT(X,Y,Z,t,sigma,r,b)

##### Exercise 8

Solve the Lorenz equations for a Prandtl number $\sigma=10$, $b=8/3$ and a scaled Rayleigh number $r=350$, starting from the initial condition ${\bf{X_0}}=(X_0,Y_0,Z_0)^T=(2,3,4)^T$. Plot the temporal evolution and 3D phase potrait of the solution $(X(t),Y(t),Z(t))^T$. Mark the fix points, you derived in [Stationary Solutions of Time-Dependent Problems](http://nbviewer.ipython.org/urls/github.com/daniel-koehn/Differential-equations-earth-system/tree/master/03_Lorenz_equations/02_Stationary_solutions_of_DE.ipynb) in the 3D phase portrait. Describe and interpret the results. Compare with the previous result from exercise 8.

In [ ]:
# SET THE PARAMETERS HERE!
sigma= 
b = 

# SET THE INITIAL CONDITIONS HERE!
X0 = 
Y0 = 
Z0 =

# Set maximum integration time and sample interval dt
tmax = 8.
dt = 5e-4

# SET THE RAYLEIGH NUMBER HERE!
r =

# Solve the Equations
X, Y, Z, t = SolveLorenz(tmax, dt, X0, Y0, Z0, sigma,r,b)

# and Visualize as a time series
PlotLorenzXvT(X,Y,Z,t,sigma,r,b)

# and as a 3-D phase portrait
PlotLorenz3D(X,Y,Z,sigma,r,b)

## What we learned:

- How to solve the Lorenz equations using a simple finite-difference scheme. 

- How to visualize the solution of ordinary differential equations using the temporal evolution and phase portrait.

- Exporing the dynamic of non-linear differential equations and the sensitivity of small changes of the initial conditions to the long term evolution of the system.

- Why physicists can only predict the time evolution of complex dynamical systems to some extent.